In [1]:
from pathlib import Path
import sys  
import os
import scipy as sp
import pandas as pd

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

import numpy as np
import pandas as pd

In [5]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [17]:
URM_train = sp.sparse.load_npz('URM_train.npz')
URM_validation = sp.sparse.load_npz('URM_validation.npz')

In [18]:
URM_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1411686 stored elements and shape (35736, 38121)>

In [7]:
class TopPopRecommenderUnseen(object):

    def fit(self, URM_train):
        
        self.URM_train = URM_train

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)
    
    
    def recommend(self, user_id, at=10, remove_seen=True):

        if remove_seen:
            seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id+1]]
            
            # remove from the popular items all the items already seen
            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert = True)
            
            #apply the mask
            unseen_items = self.popular_items[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]
            

        return recommended_items

In [8]:
topPopRecommenderUnseen = TopPopRecommenderUnseen()
topPopRecommenderUnseen.fit(URM_all)

In [9]:
topPopRecommenderUnseen.recommend(1000)

array([11146, 25392,  4601, 25415, 23712, 24388, 11148, 11079,  3062,
        3670])

In [10]:
slim_model.recommend(1000, 10)

NameError: name 'slim_model' is not defined

In [42]:
data_target_users = pd.read_csv("../data/data_target_users_test.csv")

In [43]:
URM_all_pd = pd.read_csv("../data/data_train.csv")

In [44]:
train_users = URM_all_pd['user_id'].unique()
val_users = 

In [45]:
target_users = data_target_users['user_id'].unique()

In [46]:
target_users in train_users

ValueError: operands could not be broadcast together with shapes (35736,) (34231,) 

In [49]:
np.setdiff1d(train_users, target_users)

1505

In [51]:
np.array([
		slim_model.recommend(user_id, cutoff=10) for user_id in target_users
	])

array([[ 7547,   454,  8661, ...,  6348, 22472, 14931],
       [ 7010, 11699,  6348, ..., 19114,  3077,  3130],
       [ 2774, 22658, 23377, ..., 22714, 13252, 12769],
       ...,
       [38027, 37739, 36567, ..., 37321, 37623, 36263],
       [37550, 36880, 34994, ..., 37301, 35093, 37803],
       [37193, 36778, 37805, ..., 37657, 36493, 37445]])

In [11]:
def write_submission_lib_model(trained_model, train_users, filename: str = "submission.csv") -> None:
    """Builds the submission file from a trained recommender model. The file is saved in a CSV format.

    :param trained_model: A fitted recommender model
    :type trained_model: RecommenderModel
    :param filename: The filename of the submission for this particular recommender model
    :type filename: str
    """
    target_users_test = pd.read_csv("../data/data_target_users_test.csv").to_numpy().ravel()

    recommendations = []

    for user_id in target_users_test: 
        if user_id in train_users:
            recommendations.append(trained_model.recommend(user_id, cutoff=10))
        else:
            recommendations.append(topPopRecommenderUnseen.recommend(user_id))
            print(f"User: {user_id} is new")
    
    recommendations = np.array(recommendations)

    if not os.path.exists("../submissions"):
        os.makedirs("../submissions")
    with open(f"../submissions/{filename}", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, recs in zip(target_users_test, recommendations):
            f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [13]:
URM_all_pd = pd.read_csv("../data/data_train.csv")
train_users = URM_all_pd['user_id'].unique()

In [16]:
slim_model = SLIMElasticNetRecommender(URM_all)
slim_model.load_model('result_experiments/SLIM/', 'slim_300_2_weights')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_300_2_weights'
SLIMElasticNetRecommender: Loading complete


In [17]:
write_submission_lib_model(slim_model, train_users, 'slim_urm_300_2_cold.csv')

In [21]:
len(train_users) 

35736

In [19]:
target_users_test = pd.read_csv("../data/data_target_users_test.csv").to_numpy().ravel()